In [1]:
import glob
path =r'H:\REDDIT\DATA\reddit-top-2.5-million\data' # use your path
allFiles = glob.glob(path + "/*.csv")

In [2]:
# select 500 random files
import random
trainFiles = random.sample(range(len(allFiles)), 500)

In [3]:
import pandas
frame = pandas.DataFrame()
list_ = []
for file_ in trainFiles:
    df = pandas.read_csv(allFiles[file_],index_col=None, header=0)
    list_.append(df)
frame = pandas.concat(list_)

In [4]:
len(trainFiles)

500

In [5]:
# train data set
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
X_train_counts = count_vect.fit_transform(frame['title'])
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_counts_tf = tf_transformer.transform(X_train_counts)
X_train_out = frame['score']

In [6]:
from sklearn.tree import DecisionTreeRegressor
reg = DecisionTreeRegressor(max_depth = 25)
reg.fit(X_train_counts_tf, X_train_out)

DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [7]:
from sklearn.metrics import mean_squared_error
X_train_predict = reg.predict(X_train_counts_tf)

In [8]:
mean_squared_error(X_train_out, X_train_predict)

143011.25334969597

In [9]:
# check mean squared error on 500 different subreddits than used for training
frame_test = pandas.DataFrame()
list_ = []
for file_ in range(len(allFiles)):
    if file_ not in trainFiles:
        df = pandas.read_csv(allFiles[file_],index_col=None, header=0)
        list_.append(df)
    if len(list_) == 500:
        break
frame_test = pandas.concat(list_)

In [10]:
X_test_out = frame_test['score']
X_test_counts = count_vect.transform(frame_test['title'])
X_test_counts_tf = tf_transformer.transform(X_test_counts)
X_test_predict = reg.predict(X_test_counts_tf)

In [11]:
mean_squared_error(X_test_out, X_test_predict)

153690.3317136321

In [12]:
# utility function for predicting score of lineNum row data in fileName
def predictScore(fileName, lineNum):
    dfp = pandas.read_csv(fileName, index_col=None, header=0)
    X_counts = count_vect.transform(dfp['title'])
    X_counts_tf = tf_transformer.transform(X_counts)
    X_predict = reg.predict(X_counts_tf)
    return X_predict[lineNum]

In [13]:
predictScore(path+"/coding.csv", 10)

160.00704355271313

In [14]:
"""
For experimenting with GridSearch
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor()
parameters = {'max_depth':(10,20,25)}
scoring_function = make_scorer(mean_squared_error, greater_is_better = False)
reg = GridSearchCV(regressor, parameters, scoring_function)
reg.fit(X_train_counts_tf, X_train_out)
"""

"\nFor experimenting with GridSearch\nfrom sklearn.grid_search import GridSearchCV\nfrom sklearn import metrics\nfrom sklearn.metrics import make_scorer\nfrom sklearn.tree import DecisionTreeRegressor\n\nregressor = DecisionTreeRegressor()\nparameters = {'max_depth':(10,20,25)}\nscoring_function = make_scorer(mean_squared_error, greater_is_better = False)\nreg = GridSearchCV(regressor, parameters, scoring_function)\nreg.fit(X_train_counts_tf, X_train_out)\n"

In [15]:
"""
one_hot = pandas.get_dummies(pd['domain'], sparse=True)
hot_header_list = one_hot.columns.values.tolist()
in1 = pd[['over_18']+ hot_header_list]
out1 = pd['score']
"""

"\none_hot = pandas.get_dummies(pd['domain'], sparse=True)\nhot_header_list = one_hot.columns.values.tolist()\nin1 = pd[['over_18']+ hot_header_list]\nout1 = pd['score']\n"